In [17]:
import ollama

llm = "llama3.1"
q = ''' who died on sept 9, 2024'''

res = ollama.chat(model=llm,
                  messages=[{"role": "system", "content":""},
                            {"role":"user", "content":q}])
res

{'model': 'llama3.1',
 'created_at': '2024-10-01T13:34:51.488472Z',
 'message': {'role': 'assistant',
  'content': "I don't have the ability to predict the future or know events that haven't occurred yet. I can only provide information up to my knowledge cutoff.\n\nSince you've asked about a date in the future (September 9, 2024), it's not possible for me to tell you who died on that specific date as of now. My training data does not extend to future events or dates beyond my knowledge cutoff."},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 5071285417,
 'load_duration': 1806386667,
 'prompt_eval_count': 20,
 'prompt_eval_duration': 142554000,
 'eval_count': 85,
 'eval_duration': 3120257000}

In [18]:
pip install duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


# First way of using Langchain to and using a decorator 

In [19]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

In [28]:
@tool("tool_browser")
def tool_browser(q : str) -> str: 
    """ Search on DuckDuckGo browser by passing input q"""
    return DuckDuckGoSearchRun().run(q)

In [21]:
print(tool_browser(q))

Famous deaths in September 2024. Learn about 42 historical figures, notable people and celebrities who died in Sep 2024 like James Earl Jones, Alberto Fujimori and Maggie Smith. ... (Baylor University 1973-77) and executive (GM Houston Rockets 1996-2006), dies at 86; Sep 9 Cecília Gáspár, Hungarian soccer midfielder and captain (28 caps; TSV ... Tommy Cash, the younger brother of Johnny Cash and a country musician in his own right, died Sept. 13 at 84. His death was announced by a statement from the Johnny Cash Museum. "This great man ... Sept. 9, 2024; Leer en español. James Earl Jones, a stuttering farm child who became a voice of rolling thunder as one of America's most versatile actors in a stage, film and television career ... FILE - James Earl Jones arrives before the 84th Academy Awards on Sunday, Feb. 26, 2012, in the Hollywood section of Los Angeles. Jones, who overcame racial prejudice and a severe stutter to become a celebrated icon of stage and screen has died at age 93. Hi

# Second way of creating a normal function and turning it into an Ollama Schema with Semantic Router 


In [22]:
pip install semantic-router

Note: you may need to restart the kernel to use updated packages.


In [23]:
# from semantic_router.utils.function_call import FunctionSchema

# def browser(q : str) -> str:
#     """ Search on DuckDuckGo browser by passing the input q"""
#     return DuckDuckGoSearchRun().run(q)

# tool_browser = FunctionSchema(browser).to_ollama()
# tool_browser

{'type': 'function',
 'function': {'name': 'browser',
  'description': 'Search on DuckDuckGo browser by passing the input q',
  'parameters': {'type': 'object',
   'properties': {'q': {'description': None, 'type': 'string'}},
   'required': []}}}

In [24]:
@tool("final_answer")
def final_answer(text : str) -> str:
    """ Returns a natural language response to the user by passing the input """
    return text


In [25]:
prompt = """
You know everything, you must answer every question from the user, you can use the list of tools provided to you.
Your goal is to provide the user with the best possible answer, including key information about the sources and tools used.

Note, when using a tool, you provide the tool name and the arguments to use in JSON format. 
For each call, you MUST ONLY use one tool AND the response format must ALWAYS be in the pattern:
```json
{"name":"<tool_name>", "parameters": {"<tool_input_key>":<tool_input_value>}}
```
Remember, do NOT use any tool with the same query more than once.
Remember, if the user doesn't ask a specific question, you MUST use the `final_answer` tool directly.

Every time the user asks a question, you take note of some keywords in the memory.
Every time you find some information related to the user's question, you take note of some keywords in the memory.

You should aim to collect information from a diverse range of sources before providing the answer to the user. 
Once you have collected plenty of information to answer the user's question use the `final_answer` tool.
"""

In [30]:
dic_tools = {"tool_browser":tool_browser, 
             "final_answer":final_answer}

str_tools = "\n".join([str(n+1)+". `"+str(v.name)+"`: "+str(v.description) for n,v in enumerate(dic_tools.values())])
prompt_tools = f"You can use the following tools:\n{str_tools}"
print(prompt_tools)

You can use the following tools:
1. `tool_browser`: Search on DuckDuckGo browser by passing input q
2. `final_answer`: Returns a natural language response to the user by passing the input


In [31]:
from pprint import pprint

llm_res = ollama.chat(
    model=llm,
    messages=[{'role':'system', 'content': prompt+"\n"+prompt_tools},
              {'role':'user', 'content': "hello"}]
)

print(llm_res)

{'model': 'llama3.1', 'created_at': '2024-10-01T13:43:39.524291Z', 'message': {'role': 'assistant', 'content': '{"name":"final_answer", "parameters":{"q":"Hello, I\'m here to help. How can I assist you today?"}}'}, 'done_reason': 'stop', 'done': True, 'total_duration': 3256379000, 'load_duration': 1056977875, 'prompt_eval_count': 298, 'prompt_eval_duration': 1177590000, 'eval_count': 28, 'eval_duration': 1016736000}


In [33]:
from pprint import pprint

llm_res = ollama.chat(
    model=llm,
    messages=[{'role':'system', 'content': prompt+"\n"+prompt_tools},
              {'role':'user', 'content': q}], format='json'
)

llm_res["message"]["content"]

'{"name":"tool_browser", "parameters": {"q":"\\"September 9 2024 death\\""}}'

In [36]:
import json

tool_input = json.loads(llm_res["message"]["content"])["parameters"]["q"]
print("tool input :", tool_input)

context = tool_browser(tool_input)
print("tool_output: ", context)

from pprint import pprint

llm_output = ollama.chat(
    model=llm,
    messages=[{'role':'system', 'content': "Give us the most accurate answer using the following information: \n"+context},
              {'role':'user', 'content': "q"}]
)

print("\nllm_output:\n", llm_output["message"]["content"])

tool input : "September 9 2024 death"
tool_output:  No good DuckDuckGo Search Result was found

llm_output:
 It looks like you didn't provide any information for me to work with. The phrase "No good DuckDuckGo Search Result was found" seems to be a message that indicates a search query failed or returned no relevant results, but it's not clear what specific question or topic we're discussing.

If you'd like to share more context or clarify your question, I'd be happy to try and help!
